In [10]:
import random
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
import time
import pickle
import requests
from bs4 import BeautifulSoup
import os 
import re
import pickle
import random
import nltk
import string
from transformers import AutoTokenizer
import stanza
from rapidfuzz import fuzz
from collections import Counter
#best working tokenizer
nlp = stanza.Pipeline(
    lang="nl",
    processors="tokenize",
    tokenize_no_ssplit=False,
    tokenize_mwt=False,
    verbose=False
)

In [11]:
with open("link_tex.pkl", "rb") as b:
        link_tex=pickle.load(b)


In [12]:

def clean_link_text(t):
    """Return cleaned link text or None if invalid."""
    if t is None:
        return None
    stripped = t.strip()
    if stripped == "" or all(ch in string.punctuation for ch in stripped):
        return None
    return stripped


def extract_link_sentences(html: str,nlp,count,label: str = "REFERENCE"):
    soup = BeautifulSoup(html, "html.parser")
    container = soup.find("div", {"class": "tekst"})
    if not container:
        return []

    text = container.get_text(" ")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sentences]

    # ---- collect valid link texts ----
    links = []
    for a in soup.find_all("a", {"class": "popuplink"}):
        link_text = clean_link_text(a.get_text(strip=True))
        if link_text:
            links.append(link_text)

    if not links:
        return [],[]
    output_ref = []
    output_not = []

    for sent in sentences:
        raw_entities = []

        # ---- collect all possible matches first ----
        pos=[]
        i=0
        search_pos = 0  
        for link in links:
            # exact matches
            pattern = re.compile(re.escape(link))
            match = pattern.search(sent, search_pos)
            if match:
                raw_entities.append((match.start(), match.end(), label))
                search_pos = match.end()  
                pos.append(i)

            i+=1

        if not raw_entities:
            output_not.append((sent, {"entities": []},count))
            continue

        # ---- enforce ordering rule ----
        raw_entities.sort(key=lambda x: x[0])  # sort by start offset

        filtered_entities = []
        last_start = -1
        k=0
        d=0
        for start, end, lbl in raw_entities:
            if start > last_start:
                filtered_entities.append((start, end, lbl))
                last_start = start
                links.pop(pos[k]-d)
                d+=1
            k+=1
            # else: ignore match (starts before or at same position)

        if filtered_entities:
            output_ref.append((sent, {"entities": filtered_entities},count))
    return output_ref,output_not

In [ ]:
# we only need the html text
texts=link_tex.values()
#all found references
training_data_ref=[]
#all sentences without refrences
training_data_neg=[]
count=0

for text in texts:
    result_ref,result_neg=extract_link_sentences(text,nlp,count)
    if result_ref != []:
        training_data_ref.extend(result_ref)
        training_data_neg.extend(result_neg)
    else:
        print("empty")
    count+=1
    if count%500==0:
        print(count)
    

In [ ]:



with open("training_data.pkl", "wb") as b:
        pickle.dump(training_data_ref, b)

In [ ]:
#data spacy
import pickle
import random
import spacy
from spacy.tokens import DocBin


INPUT_FILE = "training_data.pkl"
TRAIN_OUT = "train.spacy"
DEV_OUT = "dev.spacy"
DEV_SPLIT = 0.1      
NLP_MODEL = "nl_core_news_sm"   # tokenizer


def load_data(path):
    with open(path, "rb") as f:
        return pickle.load(f)


def create_docbin(nlp, data):
    db = DocBin()
    for text, ann,_ in data:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in ann["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return db


def main():
    print("Loading data…")
    data = load_data(INPUT_FILE)

    print("Shuffling and splitting…")
    random.shuffle(data)
    split = int(len(data) * (1 - DEV_SPLIT))
    train_data = data[:split]
    dev_data = data[split:]

    print(f"Train size: {len(train_data)}")
    print(f"Dev size:   {len(dev_data)}")

    print("Loading spaCy tokenizer…")
    nlp = spacy.load(NLP_MODEL)

    print("Converting training data → train.spacy…")
    train_db = create_docbin(nlp, train_data)
    train_db.to_disk(TRAIN_OUT)

    print("Converting dev data → dev.spacy…")
    dev_db = create_docbin(nlp, dev_data)
    dev_db.to_disk(DEV_OUT)

    print("\nDone!")
    print(f"Saved: {TRAIN_OUT}, {DEV_OUT}")


if __name__ == "__main__":
    main()

In [ ]:
from typing import List, Tuple, Dict, Any
import spacy
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def merge(docs):
    final=[]
    highest=0
    cursor=0
    for doc in docs:
        text,ent,doc_id= doc
        if highest < doc_id:
            highest= doc_id
    list_text = [[] for _ in range(highest+1)]
    entities = [[] for _ in range(highest+1)]
    for doc in docs:
        text,ent,doc_id= doc
        list_text[doc_id].append(text)
        entities[doc_id].append(ent)
    for i in range(highest+1):
        merged_text=""
        merged_entity=[]
        for k in  range(len(list_text[i])):
            sentence=list_text[i][k]
            entity=entities[i][k]
            for (s, e, lab) in entity.get("entities", []):
                merged_entity.append((cursor + s, cursor + e, lab))
            merged_text= merged_text +sentence
            cursor+= len(sentence)
        
        final.append((merged_text, {"entities": merged_entity}))
        cursor=0
    return final
            



Example = Tuple[str, Dict[str, Any]]  # (text, {"entities":[(start,end,label), ...]})
LABEL = "REFERENCE"
IN_TAG = "I-REFERENCE"
OUT_TAG = "O"
examples=training_data_ref+training_data_neg
nlp = spacy.blank("nl")
def spans_to_io(doc, entities, label_name=LABEL):
    """
    IO tagging: tokens are either inside a REFERENCE span (I-REFERENCE) or outside (O).
    """
    spans = [(s, e) for (s, e, lab) in entities if lab == label_name]
    tags = [OUT_TAG] * len(doc)

    for i, tok in enumerate(doc):
        ts, te = tok.idx, tok.idx + len(tok)
        inside = any(ts < e and te > s for (s, e) in spans)  # any overlap with gold span
        if inside:
            tags[i] = IN_TAG
    return tags

def token_to_feature_text(doc, i: int) -> str:
    tok = doc[i]
    prev = doc[i-1] if i > 0 else None
    nxt  = doc[i+1] if i < len(doc)-1 else None

    parts = [
        f"w={tok.text.lower()}",
        f"shape={tok.shape_}",
        f"isupper={int(tok.is_upper)}",
        f"istitle={int(tok.is_title)}",
        f"isdigit={int(tok.is_digit)}",
        f"pref3={tok.text[:3].lower()}" if len(tok.text) >= 3 else f"pref={tok.text.lower()}",
        f"suf3={tok.text[-3:].lower()}" if len(tok.text) >= 3 else f"suf={tok.text.lower()}",
    ]

    if prev:
        parts += [f"prev={prev.text.lower()}", f"prevshape={prev.shape_}"]
    else:
        parts += ["BOS=1"]

    if nxt:
        parts += [f"next={nxt.text.lower()}", f"nextshape={nxt.shape_}"]
    else:
        parts += ["EOS=1"]

    return " ".join(parts)

def build_token_rows(docs: List[Example], nlp, batch_size=128):
    """
    Build token-level dataset:
      X: feature-string per token
      y: IO tag per token
    Also returns tokenized docs (so we can reconstruct spans later).
    """
    X_text, y = [], []
    texts = [t for (t, _) in docs]
    anns  = [a for (_, a) in docs]

    parsed_docs = []
    for doc, ann in zip(nlp.pipe(texts, batch_size=batch_size), anns):
        tags = spans_to_io(doc, ann.get("entities", []))
        parsed_docs.append((doc, tags, ann.get("entities", [])))  # keep gold for eval
        for i in range(len(doc)):
            X_text.append(token_to_feature_text(doc, i))
            y.append(tags[i])

    return X_text, y, parsed_docs

def io_tags_to_spans(doc, tags, label=LABEL):
    """
    Convert consecutive I-REFERENCE tokens into character-level spans.
    """
    spans = []
    start = None

    for i, tag in enumerate(tags):
        if tag == IN_TAG and start is None:
            start = doc[i].idx
        if tag != IN_TAG and start is not None:
            end = doc[i-1].idx + len(doc[i-1])
            spans.append((start, end, label))
            start = None

    if start is not None:
        end = doc[-1].idx + len(doc[-1])
        spans.append((start, end, label))

    return spans

def span_f1_exact(gold_spans, pred_spans):
    """
    Strict span match: (start,end,label) must match exactly.
    """
    gold = set(gold_spans)
    pred = set(pred_spans)

    tp = len(gold & pred)
    fp = len(pred - gold)
    fn = len(gold - pred)

    prec = tp / (tp + fp) if (tp + fp) else 0.0
    rec  = tp / (tp + fn) if (tp + fn) else 0.0
    f1   = (2 * prec * rec / (prec + rec)) if (prec + rec) else 0.0
    return prec, rec, f1

# -------------------------
# YOUR INPUT HERE:
# examples: List[Example] = ...
# -------------------------
print("part1")
docs_6600 = merge(examples)
print("part2")
# Split by document to reduce leakage
train_docs, test_docs = train_test_split(docs_6600, test_size=0.2, random_state=42)

# Expand to token rows AFTER split (important)
X_train, y_train, _ = build_token_rows(train_docs, nlp, batch_size=128)
X_test,  y_test,  parsed_test = build_token_rows(test_docs,  nlp, batch_size=128)


with open("LG_x_train.pkl", "wb") as b:
        pickle.dump(X_train, b)
with open("LG_y_train.pkl", "wb") as b:
        pickle.dump(y_train, b)
with open("LG_x_test.pkl", "wb") as b:
    pickle.dump(X_test, b)
with open("LG_y_test.pkl", "wb") as b:
    pickle.dump(y_test, b)



In [ ]:
#data roberta
import pickle
with open("training_data.pkl", "rb") as f:
    TRAIN_DATA=pickle.load(f)

from transformers import (
    RobertaTokenizerFast,
    RobertaForTokenClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
import torch

def preprocess_spacy_dataset(dataset, tokenizer, label2id, max_length=256):
    input_ids = []
    attention_masks = []
    labels = []

    for text, ann,_ in dataset:
        entities = ann["entities"]

        encoding = tokenizer(
            text,
            return_offsets_mapping=True,
            truncation=True,
            padding="max_length",
            max_length=max_length
        )

        offsets = encoding["offset_mapping"]
        token_labels = ["O"] * len(offsets)

        # Assign BIO labels
        for start, end, ent_label in entities:
            for i, (tok_start, tok_end) in enumerate(offsets):
                if tok_start == tok_end:
                    continue  # special tokens

                if tok_start >= start and tok_end <= end:
                    if tok_start == start:
                        token_labels[i] = f"B-{ent_label}"
                    else:
                        token_labels[i] = f"I-{ent_label}"

        # Convert to label IDs
        label_ids = []
        for lab, (tok_start, tok_end) in zip(token_labels, offsets):
            if tok_start == tok_end:
                label_ids.append(-100)
            else:
                label_ids.append(label2id[lab])

        input_ids.append(encoding["input_ids"])
        attention_masks.append(encoding["attention_mask"])
        labels.append(label_ids)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels,
    }
LABEL_LIST = ["O", "B-REFERENCE", "I-REFERENCE"]
label2id = {l: i for i, l in enumerate(LABEL_LIST)}
id2label = {i: l for l, i in label2id.items()}

MODEL_NAME = "pdelobelle/robbert-v2-dutch-base"
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_NAME)
processed = preprocess_spacy_dataset(
    TRAIN_DATA,
    tokenizer,
    label2id,
    max_length=256
)
dataset = Dataset.from_dict(processed)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
with open("roberta_train.pkl", "wb") as b:
        pickle.dump(train_dataset, b)
with open("roberta_test.pkl", "wb") as b:
        pickle.dump(eval_dataset, b)